In [42]:
# load packages
import pandas as pd
import numpy as np
import folium
import altair as alt
from folium import plugins

In [43]:
## load initial raw data

# read in zillow raw data
raw_df = pd.read_csv("../data/zillowRawData.csv")
# remove unwanted column
raw_df = raw_df.drop(['RegionID', 'SizeRank', 'RegionType', 'StateName', 'Metro'], axis=1)
# rename column
raw_df = raw_df.rename(columns={'RegionName':'Zipcode', 'CountyName':'County'})

# read in US zipcode data
uszips_df = pd.read_csv("../data/uszips.csv")
# only keep necessary columns(zipcodes, lat, lng)
zip_lat_lng = uszips_df.iloc[:,:3]
# rename zip to zipcode
zip_lat_lng = zip_lat_lng.rename(columns={'zip':'Zipcode'})

In [44]:
## merge raw data with geo data

# merge latitude and longitude according to zipcode
merge_df = pd.merge(raw_df,zip_lat_lng,on='Zipcode',how='left')

# check if 'Zipcode' has any duplicates
is_unique = not merge_df['Zipcode'].duplicated().any()
if is_unique:
    print("The Zipcode column has unique values.")
else:
    print("The Zipcode column has duplicate values.")

The Zipcode column has unique values.


In [46]:
## reshape data with melting

#convert date and value data from wide to tall
merge_df = merge_df.melt(id_vars=['Zipcode','lat','lng','State','City','County'],var_name='date',value_name='value')

merge_df.head()

,Zipcode,lat,lng,State,City,County,date,value
0,77494,29.74566,-95.82302,TX,Katy,Fort Bend County,2000-01-31,208752.177188
1,8701,40.07635,-74.20311,NJ,Lakewood,Ocean County,2000-01-31,133799.965954
2,77449,29.83674,-95.73547,TX,Katy,Harris County,2000-01-31,102327.332899
3,11368,40.74962,-73.85264,NY,New York,Queens County,2000-01-31,147672.475058
4,77084,29.82698,-95.66120,TX,Houston,Harris County,2000-01-31,100957.722364


In [47]:
## visualization
m = folium.Map(location=[39.8283, -98.5795],zoom_start=4)

marker_cluster = plugins.MarkerCluster().add_to(m)

#store all zipcodes latitude and longitude
#redundant won't be plot in the following for loop
locations = merge_df[['lat','lng']]
locationlist = locations.values.tolist()

#store each zipcode list of dates and corresponding values
#the list will be reset for every zipcode in the foor loop
date_value_df = pd.DataFrame(index=range(len(locationlist)),columns=['date','value'])

m

In [48]:
locations

,lat,lng
0,29.74566,-95.82302
1,40.07635,-74.20311
2,29.83674,-95.73547
3,40.74962,-73.85264
4,29.82698,-95.66120
...,...,...
7589947,38.34413,-78.26151
7589948,40.91592,-85.27923
7589949,43.21976,-92.10315
7589950,39.49307,-80.25644


In [49]:
# Count unique combinations of column1 and column2
unique_combinations_count = merge_df.groupby(['Zipcode']).size().reset_index(name='count')

print(unique_combinations_count)

       Zipcode  count
0         1001    288
1         1002    288
2         1005    288
3         1007    288
4         1008    288
...        ...    ...
26349    99827    288
26350    99833    288
26351    99835    288
26352    99901    288
26353    99929    288

[26354 rows x 2 columns]
